In [1]:
# importation des package
import pandas as pd
import json

from openai import OpenAI
import pandas as pd
import re
import time

# lecture du fichier Excel avec quelques ligne selectionner
cols= ["Service", "Motif de consultation", "Diagnostic","Sexe","Etat-civil","Température","FC","FR","TA Systolique","TA Diastolique"]
data_base= pd.read_excel("./Fiche de consultation reserach_ML.xlsx",  skiprows=0, usecols=cols).dropna().reset_index(drop=True)
data_base = data_base[(data_base["Service"] == "PEDIATRIE GENERALE") | (data_base["Service"] == "MEDECINE INTERNE") | (data_base["Service"] == "OBSTETRIQUE") | (data_base["Service"] == "GYNECOLOGIE") | (data_base["Service"] == "GYNECOLOGIE")].reset_index(drop=True)
# data_base.head(50)

In [2]:
#data_base.to_excel('bbbb.xlsx')

In [3]:
from nltk.stem import WordNetLemmatizer 
import nltk 
import string
from spacy.lang.en.stop_words import STOP_WORDS

def clean_txt(docs):
    lemmatizer = WordNetLemmatizer() 
    # Correct mispelled words
    #spell = SpellChecker()
    #spellcorrection = [spell.correction(x) for x in docs]
    # split into words
    speech_words = nltk.word_tokenize(docs)
    # convert to lower case
    lower_text = [w.lower() for w in speech_words]
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    stripped = [re_punc.sub('', w) for w in lower_text]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    words = [w for w in words if not w in  list(STOP_WORDS)]
    #Stemm all the words in the sentence
    lem_words = [lemmatizer.lemmatize(word) for word in words]
    combined_text = ' '.join(lem_words)

    return combined_text

data_base["Diagnostic"] = data_base["Diagnostic"].apply(clean_txt)
data_base["Motif de consultation"] = data_base["Motif de consultation"].apply(clean_txt)
data_base["Etat-civil"] = data_base["Etat-civil"].apply(clean_txt)
data_base["Service"] = data_base["Service"].apply(clean_txt)




<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
data_base

,Sexe,Etat-civil,Service,Motif de consultation,Température,FC,FR,TA Systolique,TA Diastolique,Diagnostic
0,M,célibataire,medecine interne,examen prenuptiaux phase,366,49,18,123,67,isthepatite b positif
1,F,marié,obstetrique,accouchement incomplet,367,84,19,115,64,accouch avec retention placentaire
2,F,marié,gynecologie,douleur du ba ventre trouble menstruels leucor...,367,87,18,129,79,trouble de regles investiguer
3,F,célibataire,pediatrie generale,eruption cutanees fievre,36,100,24,100,60,gale infection urinaire
4,F,célibataire,pediatrie generale,eruption cutanees prurijineuses pa de fievre,367,100,24,100,60,gale
...,...,...,...,...,...,...,...,...,...,...
5152,M,célibataire,pediatrie generale,toux et rhume pa de fivre pa de fievre,366,158,56,100,60,infection de voies respiratoires supérieures
5153,M,célibataire,medecine interne,émissions de selles sanguinolentes vomissement...,358,113,28,142,98,gastroenterite aiguedeshydratation plan c neur...
5154,M,célibataire,medecine interne,aptitude physique,366,86,18,117,66,apte aux travaux leger
5155,M,marié,medecine interne,douleur thoracique irradiant au do depuis cing...,365,82,18,118,78,spondylose lombaire


In [5]:
import re

correspondances = {
    r'\bft\b':'fievre typhoide',
    r'\boc\b':'oculaire',
    r'\biu\b':'infection urinaire',
    r'\biug\b':'infection uro-génitale',
    r'\bhta\b':'hypertension artérielle',
    r'\bdlr\b':'douleur',
    r'\bpalu\b':'paludisme',
    r'\btbc\b':'tubercolse',
    r'\bsd\b':'syndrome',
    r'\bw\b':'travail'
}

def remplacer_abreviation(phrase):
    for abreviation, correspondance in correspondances.items():
        phrase = re.sub(abreviation, correspondance, phrase)
    return phrase

data_base["Diagnostic"] = data_base["Diagnostic"].apply(remplacer_abreviation)

In [6]:


import pandas as pd
#df_existant = pd.read_excel("nouveau_df.xlsx")

df_existant = data_base

import pandas as pd
import numpy as np
import Levenshtein




mots_cles = ['sepsis compliqué', 'déshydratation', 'ist',
             'hépatite B', 'gastro-entérite', 'accouchement avec rétention placentaire',
            'troubles des règles', 'gale', 'infection urinaire', 'infection des voies respiratoires',
            'syndrome infectieux', 'avortement incomplet', 'anxiété', 'gastrite', 'fièvre typhoïde', 'retard de croissance', 'angine', 
            'accès palustre', 'adénopathie inguinale', 'paludisme', "travail d'accouchement", 'aménorrhée',
            'grippe', 'parasitose intestinale', 'neuropathie périphérique', 'insuffisance veineuse' 'polyarthrite rhumatoïde', 'syndrome coronarien', 'infarctus du myocarde', 'hypertrophie ventriculaire',
            'hypertension artérielle', 'pneumonie simple', 'hémorragies du premier trimestre de la grossesse', "menace d'avortement", 
            'tumeur du cerveau', 'hémorroïde externe', 'rhinite', 'stress post-traumatique', 'candidose digestive', 
            'gastro-entérite', 'mastodynie', 'sepsis', 'teigne tondante', 'toux', 'méningite bactérienne',
            'hypertension artérielle', 'gastrite chronique', 'corps sans vie', 'ivg', 'coma hépatique', 
            'syndrome grippal', 'otite', 'bronchopneumonie', 'retard de règles', 'rhinobronchite', 'pharyngite', 
            'paludisme grave', 'bouchon de cérumen', 'asthme', 'apte à tous travaux', "rupture du tendon d'achille", 
            'gastroentérite', 'candidose digestive', 'allergie', 'rougeole', 'ulcère gastroduodénal', 'salmonellose', 
            'gastro-entérite', 'rhinite allergique', 'dermatite atopique', 'infection ORL', 'pharyngite',
            'pneumonie', 'tuberculose', 'immunodépression', 'neuropathies périphériques', 'ménométrorragie', 'rénine hypocalcémie', 'hypocalcémie', 'amibiase intestinale', 'syndrome infectieux', 
            'parasitose intestinale', 'reflux gastro-œsophagien', 'anémie', 'cancer du poumon', 'ascaridiose',
            'État normal', 'rhino-trachéite', 'rhumatisme', 'migraine', 'conjonctivite', 'névralgie', 'hépatomégalie',
            'syndrome hémolytique urémique', 'gastroentérite non fébrile', 'diabète', 'rachitisme', 'rhumatisme articulaire',
            'syndrome neuropsychiatrique', 'broncho-pneumonie', 'dysfonction érectile', 'trouble érectile', 
            'bronchiolite', 'tuberculose pulmonaire', 'apte aux travaux moyens',
            'syndrome de condensation pulmonaire', 'neuropaludisme', 'rhinite allergique', 'polype nasale',
            'sciatique', 'tuberculose pulmonaire', 'amibiase intestinale', "syndrome douloureux de l'épigastre", 
            'gingivite', 'intoxication', 'psychose hallucinatoire', 'insuffisance cardiaque', 'infection urogénitale',
            'pleurésie', 'coma alcoolique', 'pharyngite', 'rhino-bronchite', 'crise asthmatiforme', 'pyélonéphrite',
            'dermatite prurigineuse', 'infection uro-digestive', 'syndrome prémenstruel', 'levurose', 'hémorragie post-partum',
            'amibiase intestinale', 'dermatite prurigineuse', 'varicelle', 'tuberculose osseuse', 'kyste', 'éclampsie',
            "menace d'accouchement prématuré", 'coma fébrile', 'neuropaludisme', 'méningo-encéphalite', 'cardiopathie',
            'insuffisance aortique', 'aménorrhée', 'lombosciatite', 'syndrome adhérentiel', 'obésité sévère', 
            'syndrome hépatite rénal', 'insuffisance rénale', 'asthme intermittent', 'grossesse évolutive',
            'rafraîchissement thérapeutique', 'grossesse arrêtée', 'algies pelviennes', 'adhérence', 'retard des règles',
            'aménorrhée', 'bronchite', 'maladie métabolique', 'hypoglycémie', "crise d'asthme", 'syndrome œdémato-ascitique',
            'adénome', 'adénocarcinome de la prostate', 'rhume', 'Diabète', 'hématopathie', 'splénomégalie tropicale',
            'ascaridiose', 'avortement', 'gastrite à H. pylori', 'hémorragie', 'coma afebrile', 'syndrome adhérenciel',
            'otite moyenne chronique', 'rétinopathie hypertensive', 'gynécomastie idiopathique pubertaire',
            'cystite', 'avortement complet', 'contusion thoracique', 'syndrome douloureux', 'asthme bronchique', 
            'pleurésie tuberculeuse', 'dermatose', 'syndrome adénoïde', 'entérite fébrile', 'rhinite', 'trachéite',
            'parasitose intestinale', 'gastrite aiguë', 'choc hypovolémique', 'rétention urinaire', 'névralgie intercostale', 
            'invagination intestinale', 'méningo-encéphalite', 'hyperuricémie', 'maladie autoimmune', 
            'parasitose intestinale', 'urétrite', 'myome utérin', 'convulsions fébriles', 'crise asthmatiforme',
            'ménométrorragie', 'rinites', 'abcès hépatique', 'laryngite', 'gastro-entérite', 'césarienne élective',
            'Syndrome cave', 'masse médiastinale', 'syndrome myomateux', 'cervicite', 'trouble électrolytique',
            'maladie amaigrissante', 'névralgie intercostale', 'prolapsus vaginal', 'syndrome cochléaire', 
            'traumatisme abdominal', 'laryngite', 'fente labiale', 'fièvre éruptive', 'anémie décompensée', 'drépanocytose',
            'malnutrition', 'covid', 'épilepsie', 'pelvipéritonite', 'staphylococcie cutanée', 
            'anémie carentielle', 'Rhumatisme articulaire', 'hyperglycémie', 'hypocalcémie', 'lipome épigastrique',
            'adénocarcinome', 'syndrome rhumatismal', 'drépanocytose', 'rhinopharyngite', 'infections des voies respiratoires supérieures',
            'hépatite virale B', 'traumatisme crânien', 'rhinite hémorragique', 'rupture utérine', 'cordon ombilical',
            'abcès', 'malaise', 'césarienne itérative', 'galactorrhée', 'rétinite', 'gastro-entérite',
            'algoménorrhée', 'hernie inguinale', 'hernie inguinale', 'bronchopneumonie', 'rhino-bronchite', 'rhinopharyngo-amygdalite',
             'menométrorragie', 'dermatite allergique', 'psoriasis', 'trichomonase intestinale','régime thérapeutique', 
             'surmenage', 'méningite', 'annexite', 'péritonite', 'staphylococcie cutanée', 'pyélonéphrite',
             'nécrobiose', 'myomateuse', 'perturbation du cycle', 'toxi-infection', 'salmonellose', 'syndrome de Guillain-Barré',
             'infection génito-urinaire', 'mycose', 'mastodynie', 'hémicrânie', 'sciatique', 'contusion musculaire thoracique',
             'intoxication alcoolique', 'allergie', 'rhinotrachéite', 'souffrance fœtale', 'trouble du cycle menstruel',
             'faux début de travail', 'toux asthmatiforme', 'hépatite virale B', 'dermatite allergique','rétinopathie',
             'anémie', 'anémie sang', 'choc émotionnel', 
            ]


import numpy as np
mots_cles = np.unique(mots_cles)
mots_cles




# Fonction pour trouver les diagnostics les plus similaires dans mots_cles
def trouver_diagnostics_similaires(diagnostic, mots_cles):
    mots_diagnostic = diagnostic.split()  # Séparer les mots du diagnostic
    n = len(mots_diagnostic)
    
    # Pour stocker les diagnostics similaires
    diagnostics_similaires = []
    meilleures_correspondances = set()
    meilleure_similarite = 0.0
    
    # Rechercher les combinaisons de mots successifs dans le diagnostic
    for i in range(n):
        for j in range(i + 1, min(i + 5, n + 1)):  # Combinaisons de 2 à 4 mots
            combinaison = ' '.join(mots_diagnostic[i:j])
            for mot_cle in mots_cles:
                sim = 1 - Levenshtein.distance(combinaison, mot_cle) / max(len(combinaison), len(mot_cle))
                if sim > meilleure_similarite:
                    meilleure_similarite = sim
                    meilleures_correspondances = {mot_cle}
                elif sim == meilleure_similarite:
                    meilleures_correspondances.add(mot_cle)
    
    # Filtrer les meilleures correspondances pour garder le plus long si un mot est contenu dans un autre
    for mot1 in meilleures_correspondances:
        is_contained = False
        for mot2 in meilleures_correspondances:
            if mot1 != mot2 and mot1 in mot2:
                is_contained = True
                break
        if not is_contained:
            diagnostics_similaires.append(mot1)
    
    return diagnostics_similaires

nouvelles_lignes = []
# Parcourir chaque ligne du DataFrame original
for index, row in df_existant.iterrows():
    diagnostics_similaires = trouver_diagnostics_similaires(row['Diagnostic'], mots_cles)
    for diag_similaire in diagnostics_similaires:
        nouvelles_lignes.append({'Service': row['Service'], 'Sexe': row['Sexe'], 'Etat-civil': row['Etat-civil'],'Température': row['Température'], 'FC': row['FC'],'FR': row['FR'], 'TA Systolique': row['TA Systolique'], 'TA Diastolique': row['TA Diastolique'],  'Motif de consultation': row['Motif de consultation'], 'Diagnostic': diag_similaire})

# Créer un DataFrame à partir des nouvelles lignes
df_nouveau = pd.DataFrame(nouvelles_lignes)

# Afficher le DataFrame résultant
df_nouveau


,Service,Sexe,Etat-civil,Température,FC,FR,TA Systolique,TA Diastolique,Motif de consultation,Diagnostic
0,medecine interne,M,célibataire,366,49,18,123,67,examen prenuptiaux phase,hépatite B
1,obstetrique,F,marié,367,84,19,115,64,accouchement incomplet,accouchement avec rétention placentaire
2,gynecologie,F,marié,367,87,18,129,79,douleur du ba ventre trouble menstruels leucor...,troubles des règles
3,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,gale
4,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,infection urinaire
...,...,...,...,...,...,...,...,...,...,...
6034,pediatrie generale,M,célibataire,366,158,56,100,60,toux et rhume pa de fivre pa de fievre,infection des voies respiratoires
6035,medecine interne,M,célibataire,358,113,28,142,98,émissions de selles sanguinolentes vomissement...,neuropaludisme
6036,medecine interne,M,célibataire,366,86,18,117,66,aptitude physique,apte aux travaux moyens
6037,medecine interne,M,marié,365,82,18,118,78,douleur thoracique irradiant au do depuis cing...,salmonellose


In [7]:
df_nouveau['Diagnostic'].value_counts().head(60)

infection urinaire                   569
paludisme                            488
grippe                               363
ist                                  321
gastrite                             303
travail d'accouchement               265
hypertension artérielle              208
syndrome infectieux                  199
fièvre typhoïde                      195
sepsis                               141
infection urogénitale                127
parasitose intestinale               100
malaise                               88
gastrite chronique                    85
syndrome grippal                      78
grossesse évolutive                   72
État normal                           71
apte à tous travaux                   67
candidose digestive                   61
grossesse arrêtée                     61
allergie                              58
pneumonie                             55
gale                                  54
infection des voies respiratoires     48
angine          

In [8]:
# df_nouveau.to_excel("df_nouveauuu.xlsx")

In [9]:
# df_nouveau = pd.read_excel("df_nouveauuu.xlsx")

In [10]:
indextodrop = []
# Parcourir les indices de la colonne 'Température'
for index in range(len(df_nouveau['Température'])):
    # Obtenir la valeur de température de la ligne actuelle
    temperature_value = df_nouveau.at[index, 'Température']
    

    try:
        # Convertir la valeur de température en nombre (float)
        temperature_float = float(temperature_value)

        # Vérifier si la conversion a réussi
        if temperature_float > 100:  # Si la température est très élevée
            # Multiplier la valeur de la température par 0.1
            df_nouveau.at[index, 'Température'] = temperature_float * 0.1
    except ValueError:
        # Gérer les valeurs qui ne peuvent pas être converties en nombre
        print(f"Impossible de convertir la valeur de température à l'index {index}: {temperature_value}")
        indextodrop.append(index)
        
# Afficher le DataFrame mis à jour
df_nouveau = df_nouveau.drop(indextodrop)
df_nouveau

Impossible de convertir la valeur de température à l'index 23: 36;6
Impossible de convertir la valeur de température à l'index 1474: -
Impossible de convertir la valeur de température à l'index 1475: -
Impossible de convertir la valeur de température à l'index 1653: 37 ;5
Impossible de convertir la valeur de température à l'index 4258: Xxx
Impossible de convertir la valeur de température à l'index 4356: -
Impossible de convertir la valeur de température à l'index 4783: -
Impossible de convertir la valeur de température à l'index 5329: 40, 4
Impossible de convertir la valeur de température à l'index 5751: A36,6


,Service,Sexe,Etat-civil,Température,FC,FR,TA Systolique,TA Diastolique,Motif de consultation,Diagnostic
0,medecine interne,M,célibataire,36.6,49,18,123,67,examen prenuptiaux phase,hépatite B
1,obstetrique,F,marié,36.7,84,19,115,64,accouchement incomplet,accouchement avec rétention placentaire
2,gynecologie,F,marié,36.7,87,18,129,79,douleur du ba ventre trouble menstruels leucor...,troubles des règles
3,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,gale
4,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,infection urinaire
...,...,...,...,...,...,...,...,...,...,...
6034,pediatrie generale,M,célibataire,36.6,158,56,100,60,toux et rhume pa de fivre pa de fievre,infection des voies respiratoires
6035,medecine interne,M,célibataire,35.8,113,28,142,98,émissions de selles sanguinolentes vomissement...,neuropaludisme
6036,medecine interne,M,célibataire,36.6,86,18,117,66,aptitude physique,apte aux travaux moyens
6037,medecine interne,M,marié,36.5,82,18,118,78,douleur thoracique irradiant au do depuis cing...,salmonellose


In [11]:
df_nouveau

,Service,Sexe,Etat-civil,Température,FC,FR,TA Systolique,TA Diastolique,Motif de consultation,Diagnostic
0,medecine interne,M,célibataire,36.6,49,18,123,67,examen prenuptiaux phase,hépatite B
1,obstetrique,F,marié,36.7,84,19,115,64,accouchement incomplet,accouchement avec rétention placentaire
2,gynecologie,F,marié,36.7,87,18,129,79,douleur du ba ventre trouble menstruels leucor...,troubles des règles
3,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,gale
4,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,infection urinaire
...,...,...,...,...,...,...,...,...,...,...
6034,pediatrie generale,M,célibataire,36.6,158,56,100,60,toux et rhume pa de fivre pa de fievre,infection des voies respiratoires
6035,medecine interne,M,célibataire,35.8,113,28,142,98,émissions de selles sanguinolentes vomissement...,neuropaludisme
6036,medecine interne,M,célibataire,36.6,86,18,117,66,aptitude physique,apte aux travaux moyens
6037,medecine interne,M,marié,36.5,82,18,118,78,douleur thoracique irradiant au do depuis cing...,salmonellose


In [12]:
df_nouveau.to_excel("df_nouveauuu.xlsx")
df_nouveau.to_csv("df_nouveauuu11.csv")

In [13]:
df_nouveau.head(60)

,Service,Sexe,Etat-civil,Température,FC,FR,TA Systolique,TA Diastolique,Motif de consultation,Diagnostic
0,medecine interne,M,célibataire,36.6,49,18,123,67,examen prenuptiaux phase,hépatite B
1,obstetrique,F,marié,36.7,84,19,115,64,accouchement incomplet,accouchement avec rétention placentaire
2,gynecologie,F,marié,36.7,87,18,129,79,douleur du ba ventre trouble menstruels leucor...,troubles des règles
3,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,gale
4,pediatrie generale,F,célibataire,36,100,24,100,60,eruption cutanees fievre,infection urinaire
5,pediatrie generale,F,célibataire,36.7,100,24,100,60,eruption cutanees prurijineuses pa de fievre,gale
6,pediatrie generale,F,célibataire,36.6,100,24,100,60,fievre domicile non chiffree toux et rhume eru...,gale
7,gynecologie,F,marié,36.7,88,18,121,67,vomissement nausee polyarthralgie anorexie sur...,syndrome infectieux
8,pediatrie generale,M,célibataire,36.6,100,24,100,60,douleur du genou gauche,ist
9,gynecologie,F,marié,36.7,107,18,123,69,polymenorrhee,avortement incomplet
